[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv('labeledTrainData.tsv', sep = '\t', encoding = "Latin-1")
data.sample(10)

,id,sentiment,review
2598,12471_7,1,I didn't know the real events when I sat down ...
23819,6836_3,0,"Please, even if you are in the worst of the mo..."
1584,11044_7,1,"Good horror movies from France are quite rare,..."
1778,3807_4,0,"Don't get me wrong, I'm a huge fan of many of ..."
6804,9130_4,0,It is true that some fans of Peter Sellers wor...
602,11960_8,1,"\Scoop\"" is also the name of a late-Thirties E..."
17677,2074_10,1,Another Channel 4 great canned long before it'...
8180,2491_8,1,"Budget limitations, time restrictions, shootin..."
5701,11290_1,0,By far the worst movie of all time. Even Yaphe...
10807,8131_1,0,Unless the title is supposed to be some kind o...


In [13]:
from collections import Counter

vocab = Counter()

for twit in data.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(10)

[('the', 287025),
 ('a', 155092),
 ('and', 152651),
 ('of', 142970),
 ('to', 132568),
 ('is', 103227),
 ('in', 85576),
 ('I', 65971),
 ('that', 64555),
 ('this', 57195)]

## 2. Preprocessing

In [3]:
# stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Dave
[nltk_data]     Tran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(10)

[('I', 65971),
 ('/><br', 50935),
 ('The', 33760),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435)]

In [4]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

print(preprocessor(''))

In [8]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [6]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [15]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [16]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.8850666666666667
confusion matrix:
 [[3293  503]
 [ 359 3345]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      3796
           1       0.87      0.90      0.89      3704

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



## 5. Export Model 

In [17]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('week3_lab','data', 'logisticRegression.pkl'), 'wb'), protocol=4)